##  tf.train.Saver()
用于保存训练好的模型，以及在程序中读取以保存的训练好的模型

使用：
保存和恢复都需要实例化一个 tf.train.Saver。  
	• saver = tf.train.Saver()  
然后，在训练循环中，定期调用 saver.save() 方法，向文件夹中写入包含了当前模型中所有可训练变量的 checkpoint 文件。

   • saver.save(sess, FLAGS.train_dir, global_step=step)  
之后，就可以使用 saver.restore() 方法，重载模型的参数，继续训练或用于测试数据。  
	• saver.restore(sess, FLAGS.train_dir)  

一次 saver.save() 后可以在文件夹中看到新增的四个文件，  

实际上每调用一次保存操作会创建后3个数据文件并创建一个检查点（checkpoint）文件，简单理解就是权重等参数被保存到 .chkp.data 文件中，以字典的形式；图和元数据被保存到 .chkp.meta 文件中，可以被 tf.train.import_meta_graph 加载到当前默认的图。
注：
	1. saver 的操作必须在 sess 建立后进行。
	2. model.ckpt 必须存在给定文件夹中，'tmp/model.ckpt' 这里至少要有一层文件夹，否则无法保存。
	3. 恢复模型时同保存时一样，是 ‘tmp/model.ckpt’，和那3个文件名都不一样。  
   4. ckpt.model_checkpoint_path：表示模型存储的位置，不需要提供模型的名字，它会去查看checkpoint文件，看看最新的是谁，叫做什么。  

### 简单模型实现

In [1]:
import tensorflow as tf  
import numpy as np  
import os  
  
#用numpy产生数据  
x_data = np.linspace(-1,1,300)[:, np.newaxis] #转置  
noise = np.random.normal(0,0.05, x_data.shape)  
y_data = np.square(x_data)-0.5+noise  
  
#输入层  
x_ph = tf.placeholder(tf.float32, [None, 1])  
y_ph = tf.placeholder(tf.float32, [None, 1])  
  
#隐藏层  
w1 = tf.Variable(tf.random_normal([1,10]))  
b1 = tf.Variable(tf.zeros([1,10])+0.1)  
wx_plus_b1 = tf.matmul(x_ph, w1) + b1  
hidden = tf.nn.relu(wx_plus_b1)  
  
#输出层  
w2 = tf.Variable(tf.random_normal([10,1]))  
b2 = tf.Variable(tf.zeros([1,1])+0.1)  
wx_plus_b2 = tf.matmul(hidden, w2) + b2  
y = wx_plus_b2  
  
#损失  
loss = tf.reduce_mean(tf.reduce_sum(tf.square(y_ph-y),reduction_indices=[1]))  
train_op = tf.train.GradientDescentOptimizer(0.1).minimize(loss)  
  
#保存模型对象saver  
saver = tf.train.Saver()  
  
#判断模型保存路径是否存在，不存在就创建  
if not os.path.exists('backup/'):  
    os.mkdir('backup/')  
  
#初始化  
with tf.Session() as sess:  
    if os.path.exists('backup/checkpoint'):         #判断模型是否存在  
        saver.restore(sess, 'backup/model.ckpt')    #存在就从模型中恢复变量  
    else:  
        init = tf.global_variables_initializer() #不存在就初始化变量  
        sess.run(init)  
  
    for i in range(1000):  
        _,loss_value = sess.run([train_op,loss], feed_dict={x_ph:x_data, y_ph:y_data})  
        if(i%50==0):  
            save_path = saver.save(sess, 'backup/model.ckpt')  
            print("迭代次数：%d , 训练损失：%s"%(i, loss_value))  

INFO:tensorflow:Restoring parameters from backup/model.ckpt
迭代次数：0 , 训练损失：0.002868192
迭代次数：50 , 训练损失：0.0027114588
迭代次数：100 , 训练损失：0.0026606754
迭代次数：150 , 训练损失：0.0026240018
迭代次数：200 , 训练损失：0.0025948144
迭代次数：250 , 训练损失：0.002568537
迭代次数：300 , 训练损失：0.002544847
迭代次数：350 , 训练损失：0.0025259333
迭代次数：400 , 训练损失：0.0025085737
迭代次数：450 , 训练损失：0.0024923694
迭代次数：500 , 训练损失：0.0024754808
迭代次数：550 , 训练损失：0.0024585258
迭代次数：600 , 训练损失：0.0024425674
迭代次数：650 , 训练损失：0.002428894
迭代次数：700 , 训练损失：0.0024166068
迭代次数：750 , 训练损失：0.002405624
迭代次数：800 , 训练损失：0.0023958134
迭代次数：850 , 训练损失：0.002386607
迭代次数：900 , 训练损失：0.002378231
迭代次数：950 , 训练损失：0.0023707196
